In [41]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import os
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [42]:
#comunicacion y camara
import socket
import threading

In [43]:
# Configuración de TensorFlow para rendimiento
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
    # Permitir crecimiento de memoria según sea necesario
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("GPU disponible para aceleración")
    except:
        print("Error al configurar GPU, usando CPU")


In [44]:
# Inicializar MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils


In [45]:
# Configuración de directorios y archivos
data_dir = "hand_gestures_data_4_1"
os.makedirs(data_dir, exist_ok=True)

# Modelo y datos de entrenamiento
model = None
# Inicializar scaler y label encoder
scaler = StandardScaler()
label_encoder = LabelEncoder()
model_file = "hand_gesture_nn_model_4_1.h5"
scaler_file = "hand_gesture_scaler_4_1.pkl"
encoder_file = "hand_gesture_encoder_4_1.pkl"
gesture_data = "gesture_data_4_1.pkl"

In [46]:
# Variables globales para estado
data = []
labels = []

# Estado del sistema
is_trained = False
is_collecting = False
current_gesture = ""
samples_collected = 0
max_samples = 100

# Control de tiempo para la recolección continua
last_sample_time = 0
sample_delay = 0.05  # 50ms entre muestras

# Temporizador para mostrar mensajes
message = ""
message_until = 0

# Para evaluación del modelo
metrics = {
    'accuracy': 0,
    'val_accuracy': 0,
    'training_time': 0
}

In [47]:
def extract_hand_landmarks(frame):
    """
    Extrae los landmarks de las manos desde un frame de video.

    Args:
        frame: Imagen capturada por la cámara (en formato BGR).

    Returns:
        tuple: Lista de landmarks normalizados (126 elementos) y booleano indicando si se detectaron manos.
    """
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    landmarks_data = []
    hands_detected = False
    
    if results.multi_hand_landmarks:
        hands_detected = True
        # Extraer landmarks de hasta dos manos
        for hand_landmarks in results.multi_hand_landmarks:
            # Dibujar landmarks en el frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = []

            # Extraer coordenadas (x,y,z) de los 21 landmarks
            for landmark in hand_landmarks.landmark:
                landmarks.extend([landmark.x, landmark.y, landmark.z])

            landmarks_data.extend(landmarks)
    
    # Normalizar para 2 manos (si solo hay una o ninguna, rellenar con ceros)
    while len(landmarks_data) < 21 * 3 * 2:  # 21 puntos * 3 coordenadas * 2 manos
        landmarks_data.append(0.0)
    
    # Limitar a exactamente 126 características (21 puntos * 3 coordenadas * 2 manos)
    landmarks_data = landmarks_data[:21 * 3 * 2]
    
    return landmarks_data, hands_detected

In [48]:
def set_message(message_text, duration=2):
    """
    Establece un mensaje para mostrar en pantalla por una duración determinada.

    Args:
        message_text: Mensaje a mostrar (str).
        duration: Duración en segundos (int o float, por defecto 2).
    """
    global message, message_until
    message = message_text
    message_until = time.time() + duration

In [49]:
def start_collection(gesture_name):
    """
    Inicia la recolección de datos para una seña específica.

    Args:
        gesture_name: Nombre de la seña a recolectar (str).
    """
    global is_collecting, current_gesture, samples_collected
    is_collecting = True
    current_gesture = gesture_name
    samples_collected = 0
    set_message(f"Mantenga la seña frente a la cámara. Recolectando '{gesture_name}'...", 3)

In [50]:
def stop_collection():
    """
    Detiene la recolección de datos.
    """
    global is_collecting, current_gesture, samples_collected
    is_collecting = False
    current_gesture = ""
    samples_collected = 0
    set_message("Recolección finalizada", 2)

In [51]:
def save_data():
    """
    Guarda los datos recolectados en disco.
    """
    global data, labels, gesture_data
    data_dict = {
        "features": data, 
        "labels": labels
        }
    with open(f"{data_dir}/{gesture_data}", "wb") as f:
        pickle.dump(data_dict, f)
    set_message(f"Datos guardados: {len(data)} muestras", 1)

In [52]:
def collect_sample(landmarks):
    """
    Recolecta una muestra de landmarks para la seña actual.

    Args:
        landmarks: Lista de landmarks extraídos (126 elementos).

    Returns:
        bool: True si se completó la recolección (max_samples alcanzado), False en caso contrario.
    """
    global data, labels, samples_collected, last_sample_time, is_collecting

    if not is_collecting:
        return False
    
    current_time = time.time()
    # Verificar si ha pasado suficiente tiempo desde la última muestra
    if current_time - last_sample_time >= sample_delay:
        data.append(landmarks)
        labels.append(current_gesture)
        samples_collected += 1
        last_sample_time = current_time

        # Guardar datos periódicamente
        if samples_collected % 10 == 0:
            save_data()
            
        # Verificar si se ha completado la recolección
        if samples_collected >= max_samples:
            stop_collection()
            return True
    return False

In [53]:
def load_data():
    """
    Carga los datos recolectados desde disco.

    Returns:
        bool: True si los datos se cargaron correctamente, False en caso contrario.
    """
    global data, labels, gesture_data
    try:
        with open(f"{data_dir}/{gesture_data}", "rb") as f:
            data_dict = pickle.load(f)
            data = data_dict["features"]
            labels = data_dict["labels"]
        set_message(f"Datos cargados: {len(data)} muestras", 2)
        return True
    except:
        set_message("No se encontraron datos previos", 2)
        return False


In [54]:
def create_neural_network(input_shape, num_classes):
    """
    Crea una red neuronal liviana para reconocimiento de gestos.

    Args:
        input_shape: Dimensión de las características de entrada (int, ej. 126).
        num_classes: Número de clases a predecir (int).

    Returns:
        Modelo de red neuronal compilado (tensorflow.keras.Model).
    """
    model = Sequential([
        # Capa de entrada con regularización para prevenir sobreajuste
        Dense(64, activation='relu', input_shape=(input_shape,), 
              kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3), # Dropout para mejorar generalización

        # Capa oculta 
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),

        # Capa de salida
        Dense(num_classes, activation='softmax')
    ])

    # Compilar con optimizador Adam y tasa de aprendizaje reducida para estabilidad
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model


In [55]:
def train_model():
    """
    Entrena el modelo de red neuronal con los datos recolectados.

    Returns:
        float: Precisión del modelo en datos de prueba.
    """
    global data, labels, scaler, label_encoder, is_trained, metrics
    if len(data) < 10:
        set_message("Se necesitan más datos para entrenar", 2)
        return 0
    
    set_message("Preparando datos para entrenamiento...", 1)

    # Convertir a arrays NumPy
    X = np.array(data)
    y = np.array(labels)

    # Codificar etiquetas
    y_encoded = label_encoder.fit_transform(y)
    
    # Dividir datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    # Normalizar datos
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    

    # Crear modelo
    num_classes = len(set(y_encoded))
    set_message(f"Entrenando modelo con {num_classes} clases...", 2)
    
    model = create_neural_network(X_train.shape[1], num_classes)
    
    # Callbacks para mejorar el entrenamiento
    callbacks = [
        EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True),

        ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=5, 
            min_lr=0.0001)
    ]

    # Medir tiempo de entrenamiento
    start_time = time.time()

    # Entrenar el modelo
    history = model.fit(
        X_train, y_train, 
        epochs=50, 
        batch_size=32, 
        validation_split=0.2,
        callbacks=callbacks, 
        verbose=1)
    
    # Calcular tiempo de entrenamiento
    training_time = time.time() - start_time
    
    # Evaluar el modelo
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    
    # Guardar métricas
    metrics.update({
        'accuracy': test_accuracy,
        'val_accuracy': max(history.history['val_accuracy']),
        'training_time': training_time,
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'report': classification_report(y_test, y_pred, target_names=label_encoder.classes_)
    })
    
    # Guardar el modelo y componentes
    model.save(model_file)
    with open(scaler_file, 'wb') as f:
        pickle.dump(scaler, f)
    with open(encoder_file, 'wb') as f:
        pickle.dump(label_encoder, f)
    
    is_trained = True
    set_message(f"Modelo entrenado. Precisión: {test_accuracy:.2f}", 3)
    
    # Imprimir reporte detallado
    print("\n--- Informe del Modelo ---")
    print(f"Precisión en datos de prueba: {test_accuracy:.4f}")
    print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")
    print("\nClasificación detallada:")
    print(metrics['report'])
    
    return test_accuracy

In [56]:
def load_saved_model():
    """
    Carga un modelo previamente entrenado desde disco.

    Returns:
        Modelo cargado (tensorflow.keras.Model) o None si falla la carga.
    """
    global scaler, label_encoder, is_trained
    try:
        # Cargar modelo, scaler y codificador de etiquetas
        model = load_model(model_file)
        #model = tf.keras.models.load_model(model_file)
        with open(scaler_file, 'rb') as f:
            scaler = pickle.load(f)
        with open(encoder_file, 'rb') as f:
            label_encoder = pickle.load(f)
        is_trained = True
        set_message("Modelo de red neuronal cargado correctamente", 2)
        return model
    except Exception as e:
        print(f"Error al cargar el modelo: {str(e)}")
        set_message("No se encontró un modelo guardado", 2)
        return None

In [57]:
def predict(landmarks, model):
    """
    Predice la seña a partir de los landmarks usando el modelo.

    Args:
        landmarks: Lista de landmarks extraídos (126 elementos).
        model: Modelo de red neuronal cargado (tensorflow.keras.Model).

    Returns:
        tuple: Nombre de la seña predicha (str) y confianza (float).
    """
    if not is_trained or model is None:
        return None, 0
    
    # Preprocesar datos
    X = np.array([landmarks])
    X_scaled = scaler.transform(X)

    # Predecir
    prediction_probs = model.predict(X_scaled, verbose=0)[0]
    prediction_idx = np.argmax(prediction_probs)
    confidence = prediction_probs[prediction_idx]
    
    # Decodificar clase
    try:
        prediction_label = label_encoder.inverse_transform([prediction_idx])[0]
    except:
        prediction_label = "Desconocido"
    
    # Solo devolver predicción si la confianza es suficiente
    if confidence >= 0.6:
        return prediction_label, confidence
    return "Desconocido", confidence


In [58]:
def get_available_gestures():
    """
    Obtiene la lista de gestos disponibles en el conjunto de datos.

    Returns:
        list: Lista de nombres de gestos únicos.
    """
    return list(set(labels))

In [59]:
#NUEVO MENSAJE QUE DICE SI EXISTE EL MDELO
def check_model_exists():
    """Verifica si el archivo del modelo existe."""
    return os.path.exists(model_file)

In [60]:
def run():
    """
    Ejecuta el sistema completo con una interfaz gráfica basada en OpenCV.
    """
    # Cargar datos y modelo si existen
    load_data()
    model = load_saved_model()
    
    # Iniciar captura de video
    cap = cv2.VideoCapture(0)
    
    #NUEVO MENSAJE QUE DICE SI EXISTE EL MDELO
    if check_model_exists():
        model = load_saved_model()
    else:
        model = None
        set_message("¡Entrena el modelo primero (Presiona T)!", 3)

    #NUEVO MUESTRO LAS SEÑAS GRABADAS
    global labels
    if not labels:
        print("No hay señas guardadas. Usa 'R' para recolectar datos.")
        return
    
    unique_gestures = list(set(labels))  # Obtener valores únicos
    print("\\n--- Señas Guardadas ---")
    for i, gesture in enumerate(unique_gestures, 1):
        print(f"{i}. {gesture}")
    
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Espejo horizontal para una experiencia más natural
        frame = cv2.flip(frame, 1)

        # Extraer landmarks
        landmarks, hands_detected = extract_hand_landmarks(frame)
        
        # Interfaz de usuario
        frame_h, frame_w, _ = frame.shape
        
        # Mostrar estado del sistema
        if is_collecting:
            progress = int((samples_collected / max_samples) * frame_w)
            cv2.rectangle(frame, (0, 0), (progress, 20), (0, 255, 0), -1)
            cv2.rectangle(frame, (0, 0), (frame_w, 20), (255, 255, 255), 1)

            # Texto de estado
            cv2.putText(frame, f"Recolectando: {current_gesture} ({samples_collected}/{max_samples})", 
                        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Recolectar muestra si se detectan manos
            if hands_detected:
                collect_sample(landmarks)
            else:
                cv2.putText(frame, "¡No se detectan manos!", 
                            (frame_w//2 - 100, frame_h//2), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        elif is_trained and hands_detected and model is not None:
            prediction, confidence = predict(landmarks, model)
            conf_color = (0, 255, 0) if confidence > 0.85 else (0, 255, 255)
            cv2.putText(frame, f"Predicción: {prediction}", 
                        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, conf_color, 2)
            cv2.putText(frame, f"Confianza: {confidence:.2f}", 
                        (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, conf_color, 2)
        
        # Mostrar mensaje temporal
        current_time = time.time()
        if current_time < message_until:
            cv2.putText(frame, message, 
                        (10, frame_h - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)


        # Mostrar instrucciones
        cv2.putText(frame, "R: Recolectar | T: Entrenar | C: Comparar | ESC: Salir", 
                    (10, frame_h - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Mostrar frame
        cv2.imshow("Reconocimiento de Señas", frame)
        
        # Capturar teclas
        key = cv2.waitKey(1)

        # ESC para salir
        if key == 27:  # ESC
            break
        
        # R para recolectar
        elif key == ord('r') and not is_collecting:
            gesture_name = input("Ingrese el nombre de la seña: ")
            if gesture_name:
                start_collection(gesture_name)

        # T para entrenar
        elif key == ord('t') and not is_collecting:
            train_model()
            model = load_saved_model()
    
    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run()

\n--- Señas Guardadas ---
1. hola
2. construir
3. chau
4. como estas
